In [9]:
from rosemary import jpt_setup; jpt_setup()


import json
import random
import re
import os
import pandas as pd
import argparse
from open_instruct.instruction_encode_templates import encode_instruction_example, encode_few_shot_example
from open_instruct.reformat_datasets import get_all_supported_datasets

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
"""
python open_instruct/reformat_datasets.py \
    --raw_data_dir data/raw_train/ \
    --output_dir data/processed/ \
    --dataset flan_v2_SirNeural
"""

raw_data_dir = 'data/raw_train/'
output_dir = 'data/processed/'
seed = 42

In [39]:
# def convert_flan_v2_data(data_dir, output_dir):
subfolder = 'flan_v2'
data_dir = os.path.join(raw_data_dir, subfolder)
output_dir = os.path.join(output_dir, subfolder)

print(data_dir)
print(output_dir)


os.makedirs(output_dir, exist_ok=True)
examples = []


with open(os.path.join(data_dir, "flan_v2_resampled_100k.jsonl"), "r") as fin:
    for line in fin:
        examples.append(json.loads(line))
        
        

len(examples), set([x['_template_type'] for x in examples])

data/raw_train/flan_v2
data/processed/flan_v2_SirNeural/flan_v2_SirNeural/flan_v2_SirNeural/flan_v2_SirNeural/flan_v2_SirNeural/flan_v2/flan_v2/flan_v2/flan_v2


(100000, {'fs_noopt', 'fs_opt', 'zs_noopt', 'zs_opt'})

In [32]:
for idx, example in enumerate(examples):
    prompt = example["inputs"]
    if not prompt.endswith("\n") and not prompt.rstrip().endswith(":"):
        prompt += "\n"
    completion = example["targets"]
    sample = {
        "dataset": "flan_v2",
        "id": f"flan_v2_{idx}",
        "messages": [
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": completion},
        ]
    }
    break
#     fout.write(json.dumps({
#         "dataset": "flan_v2",
#         "id": f"flan_v2_{idx}",
#         "messages": [
#             {"role": "user", "content": prompt},
#             {"role": "assistant", "content": completion},
#         ]
#     }) + "\n")
    

### flan_v2_SirNeural


In [12]:
from datasets import load_dataset
input_files = [x for x in os.listdir(data_dir) if x.endswith('jsonl')]
sorted(input_files)

['cot_fs_noopt_train.jsonl',
 'cot_fs_opt_train.jsonl',
 'cot_zs_noopt_train.jsonl',
 'cot_zs_opt_train.jsonl',
 'dialog_fs_noopt_train.jsonl',
 'dialog_fs_opt_train.jsonl',
 'dialog_zs_noopt_train.jsonl',
 'dialog_zs_opt_train.jsonl',
 'flan_fs_noopt_train.jsonl',
 'flan_fs_opt_train_part1.jsonl',
 'flan_zs_noopt_train.jsonl',
 'flan_zs_opt_train.jsonl',
 'niv2_fs_noopt_train.jsonl',
 'niv2_fs_opt_train.jsonl',
 'niv2_zs_noopt_train.jsonl',
 'niv2_zs_opt_train.jsonl',
 't0_fs_noopt_train.jsonl',
 't0_zs_noopt_train.jsonl',
 't0_zs_opt_train.jsonl']

In [17]:
input_path = os.path.join(data_dir, 't0_zs_noopt_train.jsonl')

ds = load_dataset('json', data_files={'train': input_path}, split='train', cache_dir=data_dir)
ds

Found cached dataset json (/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/open-instruct/scripts/data/raw_train/flan_v2_SirNeural/json/default-987f1825b8bed00f/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


Dataset({
    features: ['inputs', 'targets', 'task'],
    num_rows: 42881000
})

In [5]:

subfolder = 'flan_v2_SirNeural'
data_dir = os.path.join('data/raw_train/', subfolder)
output_dir = os.path.join('data/processed/', subfolder)

print(data_dir, '\n', output_dir)


#### 

input_files = [x for x in os.listdir(data_dir) if x.endswith('jsonl.gz')]
input_files = os.path.join(data_dir, 'cot_fs_noopt_train.jsonl')

input_path = os.path.join(data_dir, input_file)
output_path = os.path.join(output_dir, input_file.split('_train.jsonl.gz')[0]+'_data.jsonl')

if os.path.isfile(output_path):
    continue

print(f'Processing {input_file}')

ds = load_dataset('json', data_files={'train': input_path}, split='train')
# ds = ds.select(range(100))

def convert_data_fn(example, idx):
    prompt = example['inputs']
    if not prompt.endswith("\n") and not prompt.rstrip().endswith(":"):
        prompt += "\n"
    completion = example["targets"]
    return {
        'dataset': "flan_v2", 
        'id': f"flan_v2_{idx}",
        'messages': [
            {'role': 'user', 'content': prompt},
            {"role": "assistant", "content": completion},
        ]}
ds = ds.map(convert_data_fn, 
            remove_columns=["inputs", "targets", "task"], 
            with_indices=True,
            num_proc=30,
            desc=f'Convert data for {input_file}',
            keep_in_memory=True)

ds.to_json(output_path)


data/raw_train/flan_v2_SirNeural 
 data/processed/flan_v2_SirNeural


In [143]:
examples 
n = 0
with gzip.open('data/raw_train/flan_v2_SirNeural/flan_fs_opt_train_part1.jsonl.gz', 'rt') as f:
    for line in f:
        if n%100000 == 0:
            print(n)
#         json.loads(line)
#         examples.append(json.loads(line))
        n += 1

0
100000
200000



KeyboardInterrupt



In [ ]:
50 048 610

In [122]:

input_file = 'cot_fs_noopt_train.jsonl.gz'
input_path = os.path.join(data_dir, input_file)
output_path = os.path.join(output_dir, input_file.split('_train.jsonl.gz')[0]+'_data.jsonl')

ds = load_dataset('json', data_files={'train': input_path}, split='train', streaming=True)

In [125]:


def convert_data_fn(example, idx):
    prompt = example['inputs']
    if not prompt.endswith("\n") and not prompt.rstrip().endswith(":"):
        prompt += "\n"
    completion = example["targets"]
    return {
        'dataset': "flan_v2", 
        'id': f"flan_v2_{idx}",
        'messages': [
            {'role': 'user', 'content': prompt},
            {"role": "assistant", "content": completion},
        ]}


ds = ds.map(convert_data_fn, 
            remove_columns=["inputs", "targets", "task"], 
            with_indices=True,
            num_proc=30)


TypeError: IterableDataset.map() got an unexpected keyword argument 'num_proc'

In [117]:
python scripts/resample_flan_v2.py --flan_v2_data_dir data/raw_train/flan_v2_SirNeural --total_num_samples 1000000 --output_path data/processed/flan_v2_SirNeural/flan_v2_1m.jsonl


100